In [1]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib as plt
import seaborn as sns
import os
%matplotlib inline

In [2]:
# ALL STOCKS READ INTO DF

from collections import OrderedDict
series = OrderedDict()
for filename in os.listdir("datasets/Stock Prices/"):
    path = "datasets/Stock Prices/" + filename
    col1 = str(filename.lower().split(".")[0] + '_close')
    col2 = str(filename.lower().split(".")[0] + '_volume')
    series[filename] = pd.read_csv(path, parse_dates=True, header=0, index_col=0, 
                                   usecols=['Date', 'Close', 'Volume'])
    series[filename] = series[filename].rename(columns={'Date':'date','Close':col1,'Volume':col2})

In [3]:
df_stocks = pd.DataFrame()
for each in series.keys():
    df_stocks = pd.concat([df_stocks, series[each]], axis=1).dropna()
df_stocks.head()

,abb_close,abb_volume,asys_close,asys_volume,ccgi_close,ccgi_volume,cenx_close,cenx_volume,cmi_close,cmi_volume,fslr_close,fslr_volume,ge_close,ge_volume,pcrfy_close,pcrfy_volume,tsla_close,tsla_volume
Date,,,,,,,,,,,,,,,,,,
2010-06-29,17.490000,4731000,8.27,47400,45.0,0,9.07,4079400,66.239998,4498100,114.419998,2243400,14.48,114904900,12.47,467600,23.889999,18766300
2010-06-30,17.280001,4224600,8.35,12800,42.5,0,8.83,3519600,65.129997,3268500,113.830002,1937300,14.42,81568200,12.53,201000,23.830000,17187100
2010-07-01,17.590000,6328900,8.71,70400,44.5,0,8.68,3968000,64.820000,5136200,117.449997,2078200,14.12,107077700,12.43,342500,21.959999,8218800
2010-07-02,17.430000,2315300,8.59,41200,45.0,0,8.60,2444000,64.239998,3766900,120.519997,2082800,13.88,78544600,12.45,157800,19.200001,5139800
2010-07-06,17.690001,2955000,8.60,30600,37.5,0,8.61,3302200,63.570000,3913900,121.879997,2506700,13.97,77154700,12.81,247400,16.110001,6866900


In [4]:
# ALL STOCKS READ INTO DF

from collections import OrderedDict
series = OrderedDict()
for filename in os.listdir("datasets/Sentiment/"):
    path = "datasets/Sentiment/" + filename
    col1 = str(filename[4:].lower().split("_")[0] + '_sentiment')
    col2 = str(filename[4:].lower().split("_")[0] + '_volume')
    col3 = str(filename[4:].lower().split("_")[0] + '_buzz')
    series[filename] = pd.read_csv(path, parse_dates=True, header=0, index_col=0, 
                                   usecols=['Date', 'Sentiment', 'News Volume', 'News Buzz'])
    series[filename] = series[filename].rename(columns={'Date':'date','Sentiment':col1,'News Volume':col2, 'News Buzz':col3})